In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pycocotools
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
import cv2
import os

In [2]:
pwd

'/home/shakyafernando/projects/depth-anything-V2/notebooks'

In [3]:
# Define directories
image_directory = '../data/depth-anything-v2-set2/images/' # Original Images
mask_directory = '../data/depth-anything-v2-set2/masks/' # Saved masks
csv_file = "../data/csv/depth-anything-v2-updated-with-annotations.csv" # CSV file

# Load the JSON file
coco = COCO('../data/depth-anything-v2-set2/annotations/instances_default.json') # .json file

os.makedirs(mask_directory, exist_ok=True)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../data/depth-anything-v2-set2/annotations/instances_default.json'

In [ ]:
# Load the CSV file
data = pd.read_csv(csv_file)
data.head(5)
# list(data.columns)

In [ ]:
# Function to create a color mask
def apply_color_mask(mask, color):
    """
    Apply the given color to the mask.
    Parameters:
    mask : 2D numpy array
        Binary mask with 0s and 1s.
    color : tuple
        RGB tuple (e.g., (255, 0, 0) for red).
    
    Returns:
    3D numpy array
        Colored mask.
    """
    color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
    for i in range(3):  # Apply color to each channel
        color_mask[:, :, i] = mask * color[i]
    return color_mask

In [ ]:
# Get all image ids
image_ids = coco.getImgIds()
print(image_ids)

In [4]:
# Loop through each row in the CSV
for index, row in data.iterrows():
    mask_name = row["mask_name"]
    mask_path = os.path.join(mask_directory, mask_name)

    # Check if the image exists
    if os.path.exists(mask_path):
        # Open the image
        mask = Image.open(mask_path)

        # Create a new figure for each image to avoid overlapping plots
        plt.figure(figsize=(8, 8))
        
        # Plot the image
        plt.imshow(mask)

        # Extract coordinates
        Lx0, Ly0, Lx1, Ly1 = row["Lx0"], row["Ly0"], row["Lx1"], row["Ly1"]

        # Plot the coordinates
        plt.plot(
            [Lx0, Lx1],
            [Ly0, Ly1],
            marker="o",
            color="red",
            linestyle="-",
            linewidth=2,
            markersize=5,
        )

        # Display the image with points
        plt.title(f"Image: {mask_name}")
        plt.axis("off")  # Hide axes
        plt.show()
        
    else:
        print(f"Image {mask_name} not found in folder.")
        print(mask_path)


NameError: name 'data' is not defined

In [5]:
# Loops over images to get metadata
for img_id in image_ids:
    img_info = coco.loadImgs(img_id)[0]  # Receive Image ID's
    img_name = os.path.splitext(img_info['file_name'])[0]  # Receive Image names and strip .extension
    ann_ids = coco.getAnnIds(imgIds=img_id)  # Receive Annotation ID's
    anns = coco.loadAnns(ann_ids)  # Load the received Annotation ID's

    # Initialize an empty mask for the entire image (with the same dimensions as the image)
    full_mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    # Loops over annotations in each image    
    for ann in anns:
        segmentation = ann['segmentation']
        
        if isinstance(segmentation, list):  # Polygon format
            mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
            # Convert polygons to a mask
            for poly in segmentation:
                poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
                cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

        elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
            rle = segmentation
            if isinstance(rle['counts'], list):  # Uncompressed RLE
                rle = maskUtils.frPyObjects([rle], img_info['height'], img_info['width'])
            mask = maskUtils.decode(rle)  # Decode RLE to binary mask

        else:
            continue  # Skip if segmentation format is not recognized

        # Ensure mask has the same shape as full_mask by removing the extra dimension if necessary
        if len(mask.shape) == 3 and mask.shape[2] == 1:
            mask = np.squeeze(mask, axis=2)  # Remove the third dimension

        # Combine the individual mask with the full mask (add the masks together)
        full_mask = np.maximum(full_mask, mask)  # Ensures the combination of all masks
    
    # Choose a color for the mask (RGB format, e.g., red (255, 0, 0))
    mask_color = (255, 0, 0)  # Red color for the mask, you can change it to any RGB color

    # Apply the color to the combined binary mask
    colored_mask = apply_color_mask(full_mask, mask_color)

    # Display the colored mask with all annotations combined
    # plt.figure(figsize=(10, 10))
    # plt.imshow(colored_mask)
    # plt.axis('on')
    # plt.show()

    # Save the colored mask
    mask_filename = os.path.join(mask_directory, f"mask_{img_name}_{img_id}.png")
    # cv2.imwrite(mask_filename, colored_mask)  # Save the mask as an RGB image
    print(f"Image name: {img_name} | Image ID: {img_id} | Anno ID: {ann_ids} as {mask_filename}")

NameError: name 'image_ids' is not defined